In [7]:
import urllib.request
from bs4 import BeautifulSoup
import contextlib
import pandas as pd
import re
import numpy as np
import threading
import json
from nltk.stem import WordNetLemmatizer

In [2]:
repo = "spring-boot"

In [4]:
# l1 = list(df.author_login.value_counts().keys())
# l2 = list(df.closer_login.value_counts().keys())
# l = list(set(l1+l2))
# with open(repo + "_users.txt", 'w') as f:
#     f.write(json.dumps(l))

In [ ]:
with open("users/" + repo + "_users.txt") as f:
    x = f.read()
    users = json.loads(x)

In [32]:
users_contribution = dict()
org_users = []
invalid_users = []
unlucky_users = []
others = []

In [ ]:
for i, user in enumerate(users):
    print(i)
    try:    
        with contextlib.closing(urllib.request.urlopen("https://github.com/" + user)) as page:
            soup = BeautifulSoup(page.read())       
            cntrb_section = soup.find_all("h2", {"class" : "f4 text-normal mb-2"})
            if cntrb_section:
                users_contribution[user] = re.findall("[0-9,]* contribution", cntrb_section[0].text)[0][:-13]
            elif user == "ghost":
                users_contribution[user] = np.nan
            else:
                head_section = soup.find_all("a", {"class" : "pagehead-tabs-item"})
                if head_section:                
                    org_users.append(user)
                else:
                    print("some problem ouside exception for", user)
                    others.append(user)
                users_contribution[user] = np.nan
    except Exception as e:
        users_contribution[user] = np.nan
        if(str(e) == "HTTP Error 429: too many requests" or str(e) == "<urlopen error [Errno -3] Temporary failure in name resolution>"):
            unlucky_users.append(user)
        elif str(e) == "HTTP Error 404: Not Found":
            invalid_users.append(user)
        else: 
            others.append(user)
        print(str(e))
        print("user is", user)

In [34]:
invalid_users

['netroby']

In [35]:
org_users

[]

In [36]:
unlucky_users

[]

In [37]:
others

[]

In [38]:
len(users)

114

In [39]:
len(users_contribution)

114

In [10]:
with open("cntrb/" + repo + "_github_cntrb1.txt", 'w') as f:
    f.write(json.dumps(users_contribution , indent = 4))